In [2]:
import pandas as pd
import numpy as np

In [3]:
from math import sqrt
import matplotlib.pyplot as plt

Filtering

Reading file 

In [4]:
movies = pd.read_csv("C:/Users/Prashant Singh/Downloads/movies.csv")
ratings = pd.read_csv("C:/Users/Prashant Singh/Downloads/ratings.csv")

Lets check some data 

In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [7]:
# extract year
movies['year'] = movies['title'].str.extract(r'\((\d{4})\)')

# clean title (remove year part)
movies['title'] = movies['title'].str.replace(r'\s*\(\d{4}\)', '', regex=True)


In [8]:
movies.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


Now turn of generes - remove | and put , = makes in a group

In [9]:
movies['genres'] = movies.genres.str.split('|')
movies.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


Making movie matrix with genres as 0 and 1

In [10]:
#Copying movie dataframe to new dataframe to not change the first one
moviesWithGenres = movies.copy()

In [14]:
#for every index in every row (movie) iterrates 
for index, row in movies.iterrows():
    #for every index in genres array, make a column if not existed named that genre and write 1 for its value
    for genre in row['genres']:
        moviesWithGenres.at[index, genre] = 1
#naturally there are rows that doesnt have value (Nan) cause they didnt have that genre so, we fill them instead with 0 
moviesWithGenres= moviesWithGenres.fillna(0)
moviesWithGenres.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


Using drop function to delete timestamp column cause it doesnt have any use

In [16]:
ratings= ratings.drop('timestamp', axis=1)
ratings.head()

,userId,movieId,rating
0,1,296,5.0
1,1,306,3.5
2,1,307,5.0
3,1,665,5.0
4,1,899,3.5


User input rating matrix - lets make this type for rating purpose

In [17]:
#making a costume user input rating 
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
#making a dataframe from user input rating
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


In [18]:
#finding and choosing the movies that have same movie name as our input in movies dataframe and copying them to inputid variable
inputId = movies[movies['title'].isin(inputMovies['title'])]
#merging inputid(movies that the person saw) with his ratings, making the user rating matrix
inputMovies = pd.merge(inputId, inputMovies)
#deleting columns we dont need from our new dataframe
inputMovies = inputMovies.drop('genres', axis=1).drop('year', axis=1)
inputMovies

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0
5,164600,Akira,4.5


In [19]:
inputId.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
292,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",1994
1241,1274,Akira,"[Action, Adventure, Animation, Sci-Fi]",1988
1879,1968,"Breakfast Club, The","[Comedy, Drama]",1985


Keeping the movies the person saw in dataframe that we created containing all genres as 1 and 0. Making the final movies and genere matrix

In [21]:
userMovies = moviesWithGenres[moviesWithGenres['movieId'].isin(inputMovies['movieId'])]
userMovies

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
292,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",1994,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1241,1274,Akira,"[Action, Adventure, Animation, Sci-Fi]",1988,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1879,1968,"Breakfast Club, The","[Comedy, Drama]",1985,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42774,164600,Akira,"[Action, Crime, Thriller]",2016,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


cleaning the dataframe

In [22]:
#cleaning and making new set of index to use it easier
userMovies = userMovies.reset_index(drop=True)
#deleting columns we dont need to save memory by drop function and columns
userGenreTable = userMovies.drop('movieId', axis=1).drop('title', axis=1).drop('genres', axis=1).drop('year', axis=1)
userGenreTable

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
inputMovies['rating']

0    3.5
1    2.0
2    5.0
3    4.5
4    5.0
5    4.5
Name: rating, dtype: float64

Transpose the movies matrix and multiply it with user rating (matrix multiply). Making user profile

In [24]:
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
userProfile

Adventure             10.0
Animation              8.0
Children               5.5
Comedy                13.5
Fantasy                5.5
Romance                0.0
Drama                 10.0
Action                 9.0
Crime                  9.5
Thriller               9.5
Horror                 0.0
Mystery                0.0
Sci-Fi                 4.5
IMAX                   0.0
Documentary            0.0
War                    0.0
Musical                0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

taking all movies and their genre

In [25]:
#creating genretable and choosing every movie in movies with genres dataframe but setting new index to them as their movieid
genreTable = moviesWithGenres.set_index(moviesWithGenres['movieId'])
#deleting the extra columns to save memory using drop function
genreTable = genreTable.drop('movieId', axis=1).drop('title', axis=1).drop('genres', axis=1).drop('year', axis=1)
genreTable.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
genreTable.shape

(62423, 20)

Multiply the genres with user profile to achieve weighted matrix and then take the weighted average to have a single score for every movie

In [27]:
#multiply each genre with its rating from user and summation them and divide them by the summation of userprofile
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

movieId
1    0.500000
2    0.247059
3    0.158824
4    0.276471
5    0.158824
dtype: float64

Sorting

In [28]:
#sort decending with sort value function based on score
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
recommendationTable_df.head()

movieId
5018      0.735294
122787    0.723529
64645     0.723529
144324    0.723529
81132     0.723529
dtype: float64

recommendation matrix

In [29]:
movies.loc[movies['movieId'].isin(recommendationTable_df.head(10).keys())]

,movieId,title,genres,year
4614,4719,Osmosis Jones,"[Action, Animation, Comedy, Crime, Drama, Roma...",2001
4912,5018,Motorama,"[Adventure, Comedy, Crime, Drama, Fantasy, Mys...",1991
6779,6902,Interstate 60,"[Adventure, Comedy, Drama, Fantasy, Mystery, S...",2002
9177,27344,Revolutionary Girl Utena: Adolescence of Utena...,"[Action, Adventure, Animation, Comedy, Drama, ...",1999
12879,64645,The Wrecking Crew,"[Action, Adventure, Comedy, Crime, Drama, Thri...",1968
15389,81132,Rubber,"[Action, Adventure, Comedy, Crime, Drama, Film...",2010
23306,117646,Dragonheart 2: A New Beginning,"[Action, Adventure, Comedy, Drama, Fantasy, Th...",2000
25008,122787,The 39 Steps,"[Action, Adventure, Comedy, Crime, Drama, Thri...",1959
33940,144324,Once Upon a Time,"[Action, Adventure, Comedy, Crime, Drama, Roma...",2008
36115,149488,Christmas Town,"[Action, Children, Comedy, Drama, Fantasy, Thr...",2008


Collaborative Filtering

reading and entering csv files with pandas as dataframe

In [30]:
movies = pd.read_csv("C:/Users/Prashant Singh/Downloads/movies.csv")
ratings = pd.read_csv("C:/Users/Prashant Singh/Downloads/ratings.csv")

User Based recommendation system

Making costume user input and making it a dataframe(matrix)

In [31]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


finding the movies the person saw based on name in movie dataframe and keeping them with their ratings

In [34]:
#finding and choosing the movies that have same movie name as our input in movies dataframe and copying them to inputid
inputId = movies[movies['title'].isin(inputMovies['title'].tolist())]
#merging inputid(movies that the person saw) with his ratings, making the user rating matrix
inputMovies = pd.merge(inputId, inputMovies)
inputMovies

,movieId,title,genres,rating
